In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile
train_zip = '/kaggle/input/dogs-vs-cats/train.zip'
test_zip = '/kaggle/input/dogs-vs-cats/test1.zip'
train_ref = zipfile.ZipFile(train_zip,'r')
test_ref = zipfile.ZipFile(test_zip,'r')
train_ref.extractall('/kaggle/temp/')
train_ref.close()
test_ref.extractall('/kaggle/temp/')
test_ref.close()

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/temp/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
os.mkdir('/kaggle/temp/Processed/')
os.mkdir('/kaggle/temp/Processed/train')
os.mkdir('/kaggle/temp/Processed/train/dogs')
os.mkdir('/kaggle/temp/Processed/train/cats')

In [ ]:
import shutil

In [ ]:
filenames = os.listdir('/kaggle/temp/train/')
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        shutil.move('/kaggle/temp/train/'+filename,'/kaggle/temp/Processed/train/dogs')
    else:
        shutil.move('/kaggle/temp/train/'+filename,'/kaggle/temp/Processed/train/cats')

In [ ]:
len(os.listdir('/kaggle/temp/Processed/train/dogs'))

In [ ]:
len(os.listdir('/kaggle/temp/Processed/train/cats'))

In [ ]:
os.mkdir('/kaggle/temp/Processed/validation')
os.mkdir('/kaggle/temp/Processed/validation/dogs')
os.mkdir('/kaggle/temp/Processed/validation/cats')

In [ ]:
dog_filenames = os.listdir('/kaggle/temp/Processed/train/dogs')[:2500]
cat_filenames = os.listdir('/kaggle/temp/Processed/train/cats')[:2500]

In [ ]:
for filename_d in dog_filenames:
    shutil.move('/kaggle/temp/Processed/train/dogs/'+filename_d,'/kaggle/temp/Processed/validation/dogs')
for filename_c in cat_filenames:
    shutil.move('/kaggle/temp/Processed/train/cats/'+filename_c,'/kaggle/temp/Processed/validation/cats')

In [ ]:
len(os.listdir('/kaggle/temp/Processed/train/dogs'))

In [ ]:
len(os.listdir('/kaggle/temp/Processed/train/cats'))

In [ ]:
len(os.listdir('/kaggle/temp/Processed/validation/dogs'))

In [ ]:
len(os.listdir('/kaggle/temp/Processed/validation/cats'))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
model = Sequential([Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=(150,150,3)),
                   MaxPooling2D(2,2),
                   Conv2D(32,kernel_size=(3,3),activation='relu'),
                   MaxPooling2D(2,2),
                   Conv2D(64,kernel_size=(3,3),activation='relu'),
                   MaxPooling2D(2,2),
                   Flatten(),
                   Dense(512,activation='relu'),
                   Dense(1,activation='sigmoid')
                   ])


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=RMSprop(lr=0.001),loss='binary_crossentropy',metrics=['acc'])

In [ ]:
train_dir = '/kaggle/temp/Processed/train/'
test_dir = '/kaggle/temp/test1/'
val_dir = '/kaggle/temp/Processed/validation/'

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1.0/255)
test_datagen  = ImageDataGenerator(rescale = 1.0/255)

train_generator = train_datagen.flow_from_directory(train_dir,batch_size=25,class_mode='binary',target_size=(150,150))
validation_generator = test_datagen.flow_from_directory(val_dir,batch_size=25,class_mode='binary',target_size=(150,150))

In [ ]:
history = model.fit(train_generator,
                   validation_data = validation_generator,
                   steps_per_epoch=100,
                   epochs=30,
                   validation_steps=50,
                   verbose=1)

In [ ]:
test_files = {}
for dirname, _, filenames in os.walk('/kaggle/temp/test1/'):
    for filename in filenames:
        test_files[int(filename.split('.')[0])]=filename

In [ ]:
sorted_test_files= {}
for i in sorted(test_files):
    sorted_test_files[i] = test_files[i]

In [ ]:
sorted_test_files

In [ ]:
from tensorflow.keras.preprocessing import image
predictions = []
for i in range(len(sorted_test_files)):
    test_img = '/kaggle/temp/test1/'+ sorted_test_files[i+1]
    test_img = image.load_img(test_img,target_size=(150,150))
    test_img = image.img_to_array(test_img)
    test_img = np.vstack([[test_img]])
    pred = model.predict(test_img)
    predictions.append(int(pred[0][0]))

In [ ]:
len(predictions)

In [ ]:
predictions

In [ ]:
test_img1 = '/kaggle/temp/test1/'+ sorted_test_files[8]
test_img1 = image.load_img(test_img1,target_size=(150,150))
test_img1

In [ ]:
ids = [*range(1,12501)]

In [ ]:
len(predictions)

In [ ]:
sub = {'id':ids,'label':predictions}

In [ ]:
submission = pd.DataFrame(sub)

In [ ]:
pwd

In [ ]:
submission.to_csv('submission.csv')